In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

import scipy.sparse
import random
import itertools


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)

nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [2]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

## TruncatedSVD of tf-idf matrix
Note that this rank reduction is essentially the same as doing Principal Component Analysis (PCA) on the matrix A, except that PCA mean-center the columns before performing SVD. PCA loses the sparseness of the A matrix, which can make it infeasible for large lexicons.

In [3]:
%%time

#svd = TruncatedSVD(n_components=500)
svd = TruncatedSVD(n_components=500, algorithm='arpack')
tfidf_matrix = svd.fit_transform(tfidf_matrix)

Wall time: 5min 58s


In [4]:
tfidf_matrix.shape

(27885L, 500L)

# Clustering Work
focus on largest connected component on **undirected scotus**

In [5]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

## modularity on undirected scotus
"For a given division of the network's vertices into some modules, modularity reflects the concentration of edges within modules compared with random distribution of links between all nodes regardless of modules"--*Wikipedia*

In [6]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

print mod_clust.summary()

# save clusters in pandas
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 126 clusters
Wall time: 1min 38s


## walktrap on undirected scotus

In [7]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

# save clusters in pandas
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 45s


# NLP Clustering
K-Means and Gaussian Mixture Models (GMM)

## K-Means Clustering on tf-idf (truncated SVD)

### K-means clustering with K = 10

In [8]:
%%time

# set number of clusters
num_clusters = 10

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_tfidf_clusters = km.labels_.tolist()

Wall time: 39.6 s


### K-means clustering with K = 100

In [12]:
%%time

# set number of clusters
num_clusters = 100

# run kmeans
km2 = KMeans(n_clusters=num_clusters)
km2.fit(tfidf_matrix)

nlp_tfidf_clusters2 = km2.labels_.tolist()

Wall time: 2min 20s


### K-means clustering with K = 1000

In [20]:
%%time

# set number of clusters
num_clusters = 1000

# run kmeans
km3 = KMeans(n_clusters=num_clusters)
km3.fit(tfidf_matrix)

nlp_tfidf_clusters3 = km3.labels_.tolist()

Wall time: 13min 12s


## GMM Clustering on tf-idf (truncated SVD)

### GMM Clustering with K = 10

In [15]:
%%time

# set number of components
num_components = 10

# run GMM
gmm = GaussianMixture(n_components=num_components)
gmm.fit(tfidf_matrix)

gmm_clusters = gmm.predict(tfidf_matrix).tolist()
gmm_clusters = map(int, gmm_clusters)

Wall time: 9min 12s


### GMM Clustering with K = 100 

In [16]:
%%time

# set number of components
num_components = 100

# run GMM
gmm2 = GaussianMixture(n_components=num_components)
gmm2.fit(tfidf_matrix)

gmm_clusters2 = gmm2.predict(tfidf_matrix).tolist()
gmm_clusters2 = map(int, gmm_clusters2)

Wall time: 20min 56s


### GMM Clustering with K = 1000

In [24]:
%%time

# set number of components
num_components = 1000

# run GMM
gmm3 = GaussianMixture(n_components=num_components)
gmm3.fit(tfidf_matrix)

gmm_clusters3 = gmm3.predict(tfidf_matrix).tolist()
gmm_clusters3 = map(int, gmm_clusters3)

Wall time: 58min


## Compare NLP clustering (tfidf) vs graph clustering

In [25]:
#clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])
clusters = pd.DataFrame(index=op_id_to_bow_id, columns=[])

# add in communities 
clusters['mod'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['mod'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['mod'] = clusters['mod'].astype(np.int)

# add in walktrap clusters
clusters['wt'] = walktrap_clusters
clusters['wt'].fillna(max(walktrap_clusters) + 1, inplace=True)
clusters['wt'] = clusters['wt'].astype(np.int)

# add in NLP clusters
clusters['km_10'] = nlp_tfidf_clusters
clusters['km_100'] = nlp_tfidf_clusters2
clusters['km_1000'] = nlp_tfidf_clusters3

clusters['gmm_10'] = gmm_clusters
clusters['gmm_100'] = gmm_clusters2
clusters['gmm_1000'] = gmm_clusters3

clusters.to_csv("clusters_truncated_svd.csv")

In [26]:
clusters

,mod,wt,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000
145658,1,5,7,48,387,4,81,420
89370,3,294,7,42,117,4,20,625
89371,0,35,3,13,990,2,69,224
89372,0,3,7,74,4,7,23,994
89373,0,3,7,24,706,0,73,75
89374,2,4,8,82,449,0,87,678
89375,2,5,4,61,488,6,7,240
89376,2,6,2,85,670,7,10,752
89377,2,7,0,6,408,2,55,779
89378,2,7,9,85,920,7,67,619


### load cluster csv saved in current directory

In [27]:
clusters = pd.read_csv('clusters_truncated_svd.csv')
clusters

,Unnamed: 0,mod,wt,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000
0,145658,1,5,7,48,387,4,81,420
1,89370,3,294,7,42,117,4,20,625
2,89371,0,35,3,13,990,2,69,224
3,89372,0,3,7,74,4,7,23,994
4,89373,0,3,7,24,706,0,73,75
5,89374,2,4,8,82,449,0,87,678
6,89375,2,5,4,61,488,6,7,240
7,89376,2,6,2,85,670,7,10,752
8,89377,2,7,0,6,408,2,55,779
9,89378,2,7,9,85,920,7,67,619
